In [2]:
from numpy import sqrt, exp, log, pi
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime as dt
from datetime import date,datetime,timedelta
import warnings
#from scipy.interpolate import griddata
import plotly.graph_objects as go
from arch import arch_model
import nbformat
from matplotlib.colors import Normalize
%matplotlib inline
import plotly.io as pio
pio.renderers.default = "notebook_connected"
warnings.filterwarnings("ignore")

In [1]:
class IVCalculator:
    def __init__(self, ticker, option_type, oi_lower_bound):
        self.ticker = ticker
        self.option_type = option_type
        self.oi_lower_bound = oi_lower_bound
        self.S_ticker = self.get_last_closing_price()
        self.r_1y = self.get_interest_rate()
        self.data_options = self.load_options_data()
    def get_last_closing_price(self):
        """
        Downloads the latest spot price for the given ticker.
        """
        return yf.download(self.ticker)["Close"].iloc[-1].iat[-1]
    def get_interest_rate(self):
        """
        Downloads the most recent 1-year US-T yield.
        """
        return yf.download("^IRX")["Close"].iloc[-1].iat[-1] / 100
    def load_options_data(self):
        """
        Auxiliary function to load data and drop missing values. 
        """
        data = self.options_data(self.ticker, self.option_type, self.oi_lower_bound)
        return data.dropna()
    def options_data(self, ticker, option_type="call", oi_lower_bound=100):
        """
        Downloads the options data based on specified ticker, option type and
        minimum value of open interest (oi_lower_bound). Lower bound for open
        interest serves the purpose of disregarding illiquied, barely trade
        option contracts. Lowering the OI lower bound may result in lower
        ability of the algorithm to solve for the correct IV. 
        """
        asset = yf.Ticker(ticker)
        exp_dates = asset.options
        data = pd.DataFrame()

        for exp in exp_dates:
            options = asset.option_chain(exp)
            calls = options.calls
            calls["optionType"] = "call"
            puts = options.puts
            puts["optionType"] = "put"
            d = pd.concat([calls, puts])
            d["expiration"] = pd.to_datetime(exp) + pd.DateOffset(hours=23, minutes=59, seconds=59)
            data = pd.concat([data, d])

        data["dte"] = (data["expiration"] - dt.datetime.today()).dt.days + 1
        data = data[["strike", "lastPrice", "volume", "openInterest", "optionType", "dte", "impliedVolatility"]]
        data = data[data["openInterest"] > oi_lower_bound]

        if option_type == "call":
            return data[data["optionType"] == "call"]
        elif option_type == "put":
            return data[data["optionType"] == "put"]
        else:
            return data
    